In [7]:
import findspark
findspark.init('C:\Spark\spark-3.3.0-bin-hadoop3')

In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression 
from pyspark.ml.feature import HashingTF, Tokenizer 
from pyspark.sql import SparkSession


In [10]:
#Spark Örneği oluşturuyoruz
spark = SparkSession.builder.appName('ilk ornek').getOrCreate()

In [12]:
#Eğitilmek üzere textler oluşturuyoruz SPARK a ait olan harflerin labeli 1 Hadoop a ait olan harflerin label ı 0
training = spark.createDataFrame([
    (0,"s p a r k", 1.0),
    (1,"h a d o o p", 0.0),
    (2,"s p a", 1.0),
    (3,"h a d ", 0.0),
    (4,"h p a", 0.0)], ["id", "text", "label"])

In [13]:

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.01)
pipeline=Pipeline(stages=[tokenizer,hashingTF,lr])

In [14]:
#Modeli eğitiroyruz
model = pipeline.fit(training)

In [16]:
#eğittiğimiz modeli test etmek için veri oluşturuyoruz
test = spark.createDataFrame([
    (5, "s r k"),
    (6, "h d p"),
    (7, "s p"),
    (8, "h o"),
    (9, "h o d"),
    (10, "p a")], ["id","text"])

In [17]:
# SPARK a ait harfleri prediction değeri 1 Hadoop olanların 0 hem spark hem hadoop olanlar da 0 
prediction = model.transform(test)
prediction.show()

+---+-----+---------+--------------------+--------------------+--------------------+----------+
| id| text|    words|            features|       rawPrediction|         probability|prediction|
+---+-----+---------+--------------------+--------------------+--------------------+----------+
|  5|s r k|[s, r, k]|(262144,[68693,21...|[-4.0988077254032...|[0.01632163058542...|       1.0|
|  6|h d p|[h, d, p]|(262144,[89530,14...|[4.25764171988387...|[0.98604193931025...|       0.0|
|  7|  s p|   [s, p]|(262144,[146929,2...|[-3.1656366921826...|[0.04047955108386...|       1.0|
|  8|  h o|   [h, o]|(262144,[83580,20...|[4.37417068011714...|[0.98755816338157...|       0.0|
|  9|h o d|[h, o, d]|(262144,[83580,89...|[5.30640236198333...|[0.99506473959738...|       0.0|
| 10|  p a|   [p, a]|(262144,[107107,1...|[0.07988667291752...|[0.51996105360343...|       0.0|
+---+-----+---------+--------------------+--------------------+--------------------+----------+

